In [2]:
!nvidia-smi -L


GPU 0: Tesla T4 (UUID: GPU-1253a1ea-fc4c-cb78-baa0-e96f0f966cc0)


In [3]:
!apt-get update -qq
!apt-get install -qq -y nvidia-cuda-toolkit build-essential


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Extracting templates from packages: 100%
Preconfiguring packages ...
Selecting previously unselected package libdebuginfod-common.
(Reading database ... 126102 files and directories currently installed.)
Preparing to unpack .../00-libdebuginfod-common_0.186-1ubuntu0.1_all.deb ...
Unpacking libdebuginfod-common (0.186-1ubuntu0.1) ...
Selecting previously unselected package fonts-dejavu-core.
Preparing to unpack .../01-fonts-dejavu-core_2.37-2build1_all.deb ...
Unpacking fonts-dejavu-core (2.37-2build1) ...
Selecting previously unselected package fonts-dejavu-extra.
Preparing to unpack .../02-fonts-dejavu-extra_2.37-2build1_all.deb ...
Unpacking fonts-dejavu-extra (2.37-2build1) ...
Selecting previously unselected package libcupti11.5:amd64.
Preparing to unpack .../03-libcupti11.5_11.5.114~11.5.1-1ubun

In [4]:
# Download from Google Cloud mirror
!wget -q https://storage.googleapis.com/cvdf-datasets/mnist/t10k-images-idx3-ubyte.gz \
             https://storage.googleapis.com/cvdf-datasets/mnist/t10k-labels-idx1-ubyte.gz

# Check they’re here
!ls -1 t10k-*

# Unzip
!gunzip -f t10k-images-idx3-ubyte.gz t10k-labels-idx1-ubyte.gz

# Verify
!ls -1 t10k-*


t10k-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte.gz
t10k-images-idx3-ubyte
t10k-labels-idx1-ubyte


# Training scripts to generate the model params

Exporting NHWC format params

In [5]:
import torch, torch.nn as nn, torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm
import numpy as np

BATCH  = 128
EPOCHS = 5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

class TinyCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,  8, 3, padding=1)   # 1→8
        self.fc    = nn.Linear(14*14*8, 10)

    def forward(self,x):
        x = torch.relu(self.conv1(x))          # 28×28
        x = torch.max_pool2d(x, 2)             # 14×14
        x = self.fc(x.view(x.size(0), -1))
        return x

net = TinyCNN().to(DEVICE)

tr = transforms.Compose([transforms.ToTensor()])
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(".", train=True,  download=True, transform=tr),
    batch_size=BATCH, shuffle=True)
test_loader  = torch.utils.data.DataLoader(
    datasets.MNIST(".", train=False, download=True, transform=tr),
    batch_size=1000, shuffle=False)

opt = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(1, EPOCHS+1):
    net.train()
    for x,y in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}"):
        x,y = x.to(DEVICE), y.to(DEVICE)
        opt.zero_grad(); loss_fn(net(x), y).backward(); opt.step()

net.eval(); correct = total = 0
with torch.no_grad():
    for x,y in test_loader:
        x,y = x.to(DEVICE), y.to(DEVICE)
        preds = net(x).argmax(1)
        correct += (preds == y).sum().item()
        total   += y.size(0)
print(f"Test accuracy: {100*correct/total:.2f}%")

def save_raw(fname, tensor):
    tensor.cpu().contiguous().detach().numpy().astype("f").tofile(fname)

save_raw("conv1.w", net.conv1.weight.permute(0,2,3,1))
save_raw("conv1.b", net.conv1.bias)

save_raw("fc.w",  net.fc.weight)

perm = []
for h in range(14):
    for w in range(14):
        for c in range(8):
            perm.append(c*14*14 + h*14 + w)

# after permuting / re-ordering:
fc_w_reordered = net.fc.weight[:, perm]

# move to CPU, detach, convert to numpy, cast to float32, then dump
fc_w_reordered_np = fc_w_reordered.cpu().detach().numpy().astype('f')
fc_w_reordered_np.tofile("fc.w")

print("fc.w rewritten in H‑W‑C order")

save_raw("fc.b",  net.fc.bias)

print("Weights written: conv1.w/.b  fc.w/.b")


100%|██████████| 9.91M/9.91M [00:02<00:00, 4.51MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 64.6kB/s]
100%|██████████| 1.65M/1.65M [00:06<00:00, 242kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.37MB/s]
Epoch 5/5: 100%|██████████| 469/469 [00:07<00:00, 64.69it/s]


Test accuracy: 96.92%
fc.w rewritten in H‑W‑C order
Weights written: conv1.w/.b  fc.w/.b


Exporting NCHW format params

In [6]:
import torch, torch.nn as nn, torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm
import numpy as np

BATCH  = 128
EPOCHS = 5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

class TinyCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,  8, 3, padding=1)
        self.fc    = nn.Linear(14*14*8, 10)

    def forward(self,x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = self.fc(x.view(x.size(0), -1))
        return x

net = TinyCNN().to(DEVICE)
tr = transforms.Compose([transforms.ToTensor()])
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(".", train=True,  download=True, transform=tr),
    batch_size=BATCH, shuffle=True)
test_loader  = torch.utils.data.DataLoader(
    datasets.MNIST(".", train=False, download=True, transform=tr),
    batch_size=1000, shuffle=False)

opt = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(1, EPOCHS+1):
    net.train()
    for x,y in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}"):
        x,y = x.to(DEVICE), y.to(DEVICE)
        opt.zero_grad(); loss_fn(net(x), y).backward(); opt.step()
print("Training done.")

net.eval(); correct = total = 0
with torch.no_grad():
    for x,y in test_loader:
        x,y = x.to(DEVICE), y.to(DEVICE)
        preds = net(x).argmax(1)
        correct += (preds == y).sum().item()
        total   += y.size(0)
print(f"Test accuracy: {100*correct/total:.2f}%")

def save_raw(fname, tensor):
    tensor.cpu().contiguous().detach().numpy().astype("f").tofile(fname)

save_raw("conv1_nchw.w", net.conv1.weight)
save_raw("conv1_nchw.b", net.conv1.bias)
save_raw("fc_nchw.w",  net.fc.weight)
save_raw("fc_nchw.b",  net.fc.bias)


Epoch 5/5: 100%|██████████| 469/469 [00:07<00:00, 64.97it/s]


Training done.
Test accuracy: 96.95%


# CUDA scripts for **GPU**

## NHWC

In [7]:
%%bash
cat > gpu_mnist.cu <<'EOF'
#include <cstdint>
#include <cstdio>
#include <cstdlib>
#include <vector>
#include <string>
#include <iostream>

#ifdef __CUDACC__
#include <cuda_runtime.h>

#define CUDA_ASSERT(cmd) \
    do { \
        cudaError_t _err = (cmd); \
        if (_err != cudaSuccess) { \
            exit(EXIT_FAILURE); \
        } \
    } while (0)

static uint32_t reverse32(uint32_t val) {
    return (val>>24) | ((val>>8)&0xFF00) | ((val<<8)&0xFF0000) | (val<<24);
}

static std::vector<uint8_t> load_idx(const std::string& fname, int& count, int& rows, int& cols) {
    FILE* file = fopen(fname.c_str(), "rb");
    if (!file) { exit(EXIT_FAILURE); }
    uint32_t magic, n, h=1, w=1;
    fread(&magic, 4, 1, file);
    printf("Magic: 0x%08X\n", magic);
    magic = reverse32(magic);
    printf("Magic : 0x%08X\n", magic);
    fread(&n, 4, 1, file);
    count = n = reverse32(n);
    if (magic == 0x00000803) {
        fread(&h, 4, 1, file);
        rows = h = reverse32(h);
        fread(&w, 4, 1, file);
        cols = w = reverse32(w);
    } else if (magic == 0x00000801) {
        rows = cols = 1;
    } else {
        fclose(file); exit(EXIT_FAILURE);
    }
    size_t sz = (size_t)count * rows * cols;
    std::vector<uint8_t> buf(sz);
    size_t got = fread(buf.data(), 1, sz, file);
    fclose(file);
    return buf;
}

__global__ void act_relu(float* arr, int len) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < len) {
        arr[idx] = fmaxf(0.f, arr[idx]);
    }
}

__global__ void pool2x2(const float* __restrict__ src, float* dst, int B, int H, int W, int D) {
    int outH = H / 2;
    int outW = W / 2;
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int total = B * outH * outW * D;
    if (tid >= total) return;
    int d  = tid % D;
    int x2 = (tid / D) % outW;
    int y2 = (tid / D / outW) % outH;
    int b  = tid / (D * outH * outW);
    int y0 = y2 * 2, x0 = x2 * 2;
    float mx = src[((b*H + y0)*W + x0)*D + d];
    mx = fmaxf(mx, src[((b*H + y0+1)*W + x0)*D + d]);
    mx = fmaxf(mx, src[((b*H + y0)*W + x0+1)*D + d]);
    mx = fmaxf(mx, src[((b*H + y0+1)*W + x0+1)*D + d]);
    dst[tid] = mx;
}

__global__ void conv3x3(const float* __restrict__ inp,
                        const float* __restrict__ filt,
                        const float* __restrict__ bias,
                        float*       __restrict__ out,
                        int B, int H, int W, int Din, int Dout) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int total = B * H * W * Dout;
    if (tid >= total) return;
    int od = tid % Dout;
    int ox = (tid / Dout) % W;
    int oy = (tid / Dout / W) % H;
    int ob = tid / (Dout * H * W);
    float acc = bias[od];
    for (int id = 0; id < Din; ++id) {
        #pragma unroll
        for (int dy = -1; dy <= 1; ++dy) {
            for (int dx = -1; dx <= 1; ++dx) {
                int y = oy + dy;
                int x = ox + dx;
                if (y >= 0 && y < H && x >= 0 && x < W) {
                    int inp_idx = ((ob*H + y)*W + x)*Din + id;
                    int filt_idx = ((od*3 + (dy+1))*3 + (dx+1))*Din + id;
                    acc += inp[inp_idx] * filt[filt_idx];
                }
            }
        }
    }
    out[tid] = acc;
}

__global__ void dense_layer(const float* __restrict__ inp,
                           const float* __restrict__ wt,
                           const float* __restrict__ bias,
                           float*       __restrict__ out,
                           int B, int Din, int Dout) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int total = B * Dout;
    if (tid >= total) return;
    int b = tid / Dout;
    int o = tid % Dout;
    float acc = bias[o];
    for (int i = 0; i < Din; ++i) {
        acc += inp[b*Din + i] * wt[o*Din + i];
    }
    out[tid] = acc;
}

struct DeviceNet {
    static constexpr int IN_H=28, IN_W=28, IN_C=1;
    static constexpr int CONV_C=8;
    static constexpr int CONV_H=28, CONV_W=28, POOL_H=14, POOL_W=14;
    static constexpr int FCIN = POOL_H*POOL_W*CONV_C;
    static constexpr int FCOUT=10;

    int batch;
    float *g_img, *g_conv_w, *g_conv_b, *g_conv_out, *g_pool,
          *g_fc_w, *g_fc_b, *g_fc_out;

    DeviceNet(int n): batch(n) {
        auto alloc=[&](float** ptr, size_t sz, const char* label){
            CUDA_ASSERT(cudaMalloc(ptr, sz*sizeof(float)));
        };
        alloc(&g_img,     (size_t)batch*IN_H*IN_W*IN_C, "g_img");
        alloc(&g_conv_w,  (size_t)CONV_C*3*3*IN_C,  "g_conv_w"); alloc(&g_conv_b, CONV_C, "g_conv_b");
        alloc(&g_conv_out,(size_t)batch*CONV_H*CONV_W*CONV_C, "g_conv_out");  alloc(&g_pool, (size_t)batch*POOL_H*POOL_W*CONV_C, "g_pool");
        alloc(&g_fc_w,    (size_t)FCOUT*FCIN, "g_fc_w"); alloc(&g_fc_b, FCOUT, "g_fc_b");
        alloc(&g_fc_out,  (size_t)batch*FCOUT,   "g_fc_out");
        auto loadw = [&](const char* fname, float* dptr, size_t len){
            std::vector<float> buf(len);
            FILE* file = fopen(fname, "rb");
            size_t got = fread(buf.data(), sizeof(float), len, file);
            fclose(file);
            CUDA_ASSERT(cudaMemcpy(dptr, buf.data(), len*sizeof(float), cudaMemcpyHostToDevice));
        };
        loadw("conv1.w", g_conv_w, (size_t)CONV_C*3*3*IN_C);
        loadw("conv1.b", g_conv_b, (size_t)CONV_C);
        loadw("fc.w",    g_fc_w, (size_t)FCOUT*FCIN);
        loadw("fc.b",    g_fc_b, (size_t)FCOUT);
        printf("Model params\n");
    }
    ~DeviceNet() {
        cudaFree(g_img);
        cudaFree(g_conv_w);
        cudaFree(g_conv_b);
        cudaFree(g_conv_out);
        cudaFree(g_pool);
        cudaFree(g_fc_w);
        cudaFree(g_fc_b);
        cudaFree(g_fc_out);
    }
    void propagate(const uint8_t* img) {
        std::vector<float> h_img((size_t)batch*IN_H*IN_W*IN_C);
        for (size_t i = 0; i < h_img.size(); ++i) h_img[i] = img[i] / 255.f;
        CUDA_ASSERT(cudaMemcpy(g_img, h_img.data(), h_img.size()*sizeof(float), cudaMemcpyHostToDevice));
        int blk = 256;
        auto grid = [&](int n) { return dim3((n + blk - 1) / blk); };
        int conv_sz = batch*CONV_H*CONV_W*CONV_C;
        conv3x3<<<grid(conv_sz), blk>>>(g_img, g_conv_w, g_conv_b, g_conv_out, batch, IN_H, IN_W, IN_C, CONV_C);
        act_relu<<<grid(conv_sz), blk>>>(g_conv_out, conv_sz);
        int pool_sz = batch*POOL_H*POOL_W*CONV_C;
        pool2x2<<<grid(pool_sz), blk>>>(g_conv_out, g_pool, batch, CONV_H, CONV_W, CONV_C);
        int fc_sz = batch*FCOUT;
        dense_layer<<<grid(fc_sz), blk>>>(g_pool, g_fc_w, g_fc_b, g_fc_out, batch, FCIN, FCOUT);
        CUDA_ASSERT(cudaGetLastError());
    }
    double timeit(const uint8_t* img) {
        cudaEvent_t st,en; CUDA_ASSERT(cudaEventCreate(&st)); CUDA_ASSERT(cudaEventCreate(&en));
        CUDA_ASSERT(cudaEventRecord(st));
        propagate(img);
        CUDA_ASSERT(cudaEventRecord(en));
        CUDA_ASSERT(cudaEventSynchronize(en));
        float ms; CUDA_ASSERT(cudaEventElapsedTime(&ms, st, en));
        CUDA_ASSERT(cudaEventDestroy(st)); CUDA_ASSERT(cudaEventDestroy(en));
        return ms/1000.0;
    }
    int assess(const uint8_t* img, const uint8_t* lbl) {
        printf("Running forward for accuracy...\n");
        propagate(img);
        std::vector<float> h_out((size_t)batch*FCOUT);
        CUDA_ASSERT(cudaMemcpy(h_out.data(), g_fc_out, h_out.size()*sizeof(float), cudaMemcpyDeviceToHost));
        int nright = 0;
        for (int i = 0; i < batch; ++i) {
            int pred = 0; float maxv = -1e9f;
            for (int j = 0; j < FCOUT; ++j) {
                float v = h_out[i*FCOUT + j];
                if (v > maxv) { maxv = v; pred = j; }
            }
            if (pred == lbl[i]) ++nright;
        }
        printf("Accuracy done.\n");
        return nright;
    }
};

static void launch_gpu() {
    int nImg, rImg, cImg, nLbl, rLbl, cLbl;
    auto imgs = load_idx("t10k-images-idx3-ubyte", nImg, rImg, cImg);
    auto lbls = load_idx("t10k-labels-idx1-ubyte", nLbl, rLbl, cLbl);
    if (nImg != 10000 || nLbl != 10000 || nImg != nLbl) {
        return;
    }
    DeviceNet model(nImg);
    double sec = model.timeit(imgs.data());
    int top1 = model.assess(imgs.data(), lbls.data());
    double throughput = nImg / sec;
    double acc_percent = top1 * 100.0 / nImg;
    std::cout << "========================================\n";
    std::cout << "  Throughput: " << throughput << " img/s\n";
    std::cout << "  Accuracy:   " << acc_percent << "% (" << top1 << "/" << nImg << ")\n";
    std::cout << "========================================\n";
}

int main() {
    launch_gpu();
    return 0;
}

#else
#error "error"
#endif

EOF

nvcc gpu_mnist.cu -O3 -o gpu_mnist

gpu_mnist.cu(27): warning #1650-D: result of call is not used
      fread(&magic, 4, 1, file);
      ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

gpu_mnist.cu(31): warning #1650-D: result of call is not used
      fread(&n, 4, 1, file);
      ^

gpu_mnist.cu(34): warning #1650-D: result of call is not used
          fread(&h, 4, 1, file);
          ^

gpu_mnist.cu(36): warning #1650-D: result of call is not used
          fread(&w, 4, 1, file);
          ^

gpu_mnist.cu(27): warning #1650-D: result of call is not used
      fread(&magic, 4, 1, file);
      ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

gpu_mnist.cu(31): warning #1650-D: result of call is not used
      fread(&n, 4, 1, file);
      ^

gpu_mnist.cu(34): warning #1650-D: result of call is not used
          fread(&h, 4, 1, file);
          ^

gpu_mnist.cu(36): warning #1650-D: result of call is not used
          fread(&w, 4, 1, file);
          ^

gp

In [8]:
!nvcc -arch=sm_75 gpu_mnist.cu -O3 -o gpu_mnist


gpu_mnist.cu(27): warning #1650-D: result of call is not used
      fread(&magic, 4, 1, file);
      ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

gpu_mnist.cu(31): warning #1650-D: result of call is not used
      fread(&n, 4, 1, file);
      ^

gpu_mnist.cu(34): warning #1650-D: result of call is not used
          fread(&h, 4, 1, file);
          ^

gpu_mnist.cu(36): warning #1650-D: result of call is not used
          fread(&w, 4, 1, file);
          ^

gpu_mnist.cu(27): warning #1650-D: result of call is not used
      fread(&magic, 4, 1, file);
      ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

gpu_mnist.cu(31): warning #1650-D: result of call is not used
      fread(&n, 4, 1, file);
      ^

gpu_mnist.cu(34): warning #1650-D: result of call is not used
          fread(&h, 4, 1, file);
          ^

gpu_mnist.cu(36): warning #1650-D: result of call is not used
          fread(&w, 4, 1, file);
          ^

gp

In [9]:
!./gpu_mnist


Magic: 0x03080000
Magic : 0x00000803
Magic: 0x01080000
Magic : 0x00000801
Model params
Running forward for accuracy...
Accuracy done.
  Throughput: 214563 img/s
  Accuracy:   96.92% (9692/10000)


## NCHW

In [10]:
%%bash
cat > gpu_mnist_nchw.cu <<'EOF'
#include <cstdint>
#include <cstdio>
#include <cstdlib>
#include <vector>
#include <string>
#include <iostream>
#include <chrono>

#ifdef __CUDACC__
#include <cuda_runtime.h>


#define CUDA_CHECK(call) \
    do { \
        cudaError_t _e = (call); \
        if (_e != cudaSuccess) { \
            exit(EXIT_FAILURE); \
        } \
    } while (0)

static uint32_t swap32(uint32_t v) {
    return (v>>24) | ((v>>8)&0xFF00) | ((v<<8)&0xFF0000) | (v<<24);
}

static std::vector<uint8_t> read_idx(const std::string& path, int& n, int& r, int& c) {
    FILE* f = fopen(path.c_str(), "rb");
    if (!f) { exit(EXIT_FAILURE); }
    uint32_t magic, num_items, rows=1, cols=1;
    fread(&magic, 4, 1, f);
    magic = swap32(magic);
    fread(&num_items, 4, 1, f);
    n = num_items = swap32(num_items);

    if (magic == 0x00000803) {
        fread(&rows, 4, 1, f);
        r = rows = swap32(rows);
        fread(&cols, 4, 1, f);
        c = cols = swap32(cols);
    } else if (magic == 0x00000801) {
        r = c = 1;
    } else {
        fclose(f); exit(EXIT_FAILURE);
    }

    size_t data_size = (size_t)n * r * c;
    std::vector<uint8_t> data(data_size);
    size_t bytes_read = fread(data.data(), 1, data_size, f);
    if (bytes_read != data_size) { fclose(f); exit(EXIT_FAILURE); }
    fclose(f);
    return data;
}

__global__ void relu_kernel(float* x, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) {
        x[i] = fmaxf(0.f, x[i]);
    }
}

__global__ void maxpool2x2_kernel(const float* __restrict__ x, float* y, int N, int C, int H, int W) {
    int H_out = H / 2;
    int W_out = W / 2;

    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int tot = N * C * H_out * W_out;
    if (idx >= tot) return;

    int w2 = idx % W_out;
    int h2 = (idx / W_out) % H_out;
    int c  = (idx / (W_out * H_out)) % C;
    int n  = idx / (C * H_out * W_out);

    int h0 = h2 * 2, w0 = w2 * 2;

    const float* x_offset = x + (n * C + c) * H * W;
    float m = x_offset[h0 * W + w0];
    m = fmaxf(m, x_offset[(h0+1) * W + w0]);
    m = fmaxf(m, x_offset[h0 * W + w0+1]);
    m = fmaxf(m, x_offset[(h0+1) * W + w0+1]);
    y[idx] = m;
}

__global__ void conv3x3_kernel(const float* __restrict__ x,
                               const float* __restrict__ w,
                               const float* __restrict__ b,
                               float*       __restrict__ y,
                               int N, int H, int W, int Cin, int Cout) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int tot = N * Cout * H * W;
    if (idx >= tot) return;

    int w0 = idx % W;
    int h0 = (idx / W) % H;
    int co = (idx / (H * W)) % Cout;
    int n  = idx / (Cout * H * W);

    float acc = b[co];

    for (int ci = 0; ci < Cin; ++ci) {
        const float* w_offset = w + (co * Cin + ci) * 3 * 3;
        const float* x_offset = x + (n * Cin + ci) * H * W;

        for (int dr = -1; dr <= 1; ++dr) {
            for (int dc = -1; dc <= 1; ++dc) {
                int h = h0 + dr;
                int c = w0 + dc;

                if (h >= 0 && h < H && c >= 0 && c < W) {
                    int x_idx = h * W + c;
                    int w_idx = (dr+1) * 3 + (dc+1);
                    acc += x_offset[x_idx] * w_offset[w_idx];
                }
            }
        }
    }
    y[idx] = acc;
}

__global__ void fc_kernel(const float* __restrict__ x,
                          const float* __restrict__ w,
                          const float* __restrict__ b,
                          float*       __restrict__ y,
                          int N, int D, int K) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int tot = N * K;
    if (idx >= tot) return;

    int n = idx / K;
    int k = idx % K;

    float acc = b[k];

    for (int d = 0; d < D; ++d) {
        acc += x[n*D + d] * w[k*D + d];
    }
    y[idx] = acc;
}

struct GpuNet {
    static constexpr int H0=28, W0=28, C0=1;
    static constexpr int C1=8;
    static constexpr int H1=28, W1=28, H1p=14, W1p=14;
    static constexpr int FC_IN = H1p*W1p*C1;
    static constexpr int FC_OUT=10;

    int N;

    float *d_x, *d_c1w, *d_c1b, *d_y1, *d_y1p,
          *d_fc_w, *d_fc_b, *d_fc_out;

    GpuNet(int nImg): N(nImg) {

        auto mal=[&](float** p, size_t s, const char* name){
            CUDA_CHECK(cudaMalloc(p, s*sizeof(float)));
        };

        mal(&d_x,     (size_t)N*C0*H0*W0, "d_x");
        mal(&d_c1w,   (size_t)C1*3*3*C0,  "d_c1w"); mal(&d_c1b, C1, "d_c1b");
        mal(&d_y1,    (size_t)N*C1*H1*W1, "d_y1");  mal(&d_y1p, (size_t)N*C1*H1p*W1p, "d_y1p");
        mal(&d_fc_w,  (size_t)FC_OUT*FC_IN, "d_fc_w"); mal(&d_fc_b, FC_OUT, "d_fc_b");
        mal(&d_fc_out,(size_t)N*FC_OUT,   "d_fc_out");

        auto load_weights = [&](const char* prefix, char suffix, float* d_ptr, size_t num_elements) {
            char fpath[256];
            snprintf(fpath, sizeof(fpath), "%s_nchw.%c", prefix, suffix);
            FILE *f = fopen(fpath, "rb");
            if (!f) { exit(EXIT_FAILURE); }
            std::vector<float> h_buf(num_elements);
            size_t read_count = fread(h_buf.data(), sizeof(float), num_elements, f);
            fclose(f);
            if (read_count != num_elements) { exit(EXIT_FAILURE); }
            CUDA_CHECK(cudaMemcpy(d_ptr, h_buf.data(), num_elements * sizeof(float), cudaMemcpyHostToDevice));
        };

        load_weights("conv1", 'w', d_c1w, C1*C0*3*3);
        load_weights("conv1", 'b', d_c1b, C1);
        load_weights("fc",    'w', d_fc_w, FC_OUT*FC_IN);
        load_weights("fc",    'b', d_fc_b, FC_OUT);
    }

    ~GpuNet() {
        cudaFree(d_x); cudaFree(d_c1w); cudaFree(d_c1b); cudaFree(d_y1);
        cudaFree(d_y1p); cudaFree(d_fc_w); cudaFree(d_fc_b); cudaFree(d_fc_out);
    }

    void forward(const uint8_t* img) {
        std::vector<float> h_x(N * C0 * H0 * W0);
        for (size_t i = 0; i < h_x.size(); ++i) {
            h_x[i] = img[i] / 255.0f;
        }
        CUDA_CHECK(cudaMemcpy(d_x, h_x.data(), h_x.size() * sizeof(float), cudaMemcpyHostToDevice));

        int threads = 256;
        auto grid = [&](int total_elems) { return (total_elems + threads - 1) / threads; };

        conv3x3_kernel<<<grid(N*C1*H1*W1), threads>>>(
            d_x, d_c1w, d_c1b, d_y1, N, H1, W1, C0, C1);
        relu_kernel<<<grid(N*C1*H1*W1), threads>>>(d_y1, N*C1*H1*W1);
        maxpool2x2_kernel<<<grid(N*C1*H1p*W1p), threads>>>(
            d_y1, d_y1p, N, C1, H1, W1);

        fc_kernel<<<grid(N*FC_OUT), threads>>>(
            d_y1p, d_fc_w, d_fc_b, d_fc_out, N, FC_IN, FC_OUT);
    }

    int accuracy(const uint8_t* img, const uint8_t* lbl) {
        forward(img);

        std::vector<float> h_fc_out(N * FC_OUT);
        CUDA_CHECK(cudaMemcpy(h_fc_out.data(), d_fc_out, h_fc_out.size() * sizeof(float), cudaMemcpyDeviceToHost));

        int correct = 0;
        for (int i = 0; i < N; ++i) {
            float max_val = -1e9;
            int pred = -1;
            for (int j = 0; j < FC_OUT; ++j) {
                if (h_fc_out[i * FC_OUT + j] > max_val) {
                    max_val = h_fc_out[i * FC_OUT + j];
                    pred = j;
                }
            }
            if (pred == lbl[i]) {
                correct++;
            }
        }
        return correct;
    }
};

static void run_gpu() {
    int n_test, h_test, w_test, n_lbl, r_lbl, c_lbl;
    std::vector<uint8_t> test_img = read_idx("t10k-images-idx3-ubyte", n_test, h_test, w_test);
    std::vector<uint8_t> test_lbl = read_idx("t10k-labels-idx1-ubyte", n_lbl, r_lbl, c_lbl);
    if (n_test != n_lbl) { fprintf(stderr, "[ERROR] Mismatch between image count (%d) and label count (%d)\n", n_test, n_lbl); exit(EXIT_FAILURE); }

    GpuNet net(n_test);

    double sec = 0.0;
    int num_runs = 10;
    auto start = std::chrono::high_resolution_clock::now();
    for (int i = 0; i < num_runs; ++i) {
        net.forward(test_img.data());
    }
    CUDA_CHECK(cudaDeviceSynchronize());
    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> duration = end - start;
    sec = duration.count() / num_runs;

    int top1 = net.accuracy(test_img.data(), test_lbl.data());

    double throughput = n_test / sec;
    double acc_percent = top1 * 100.0 / n_test;
    std::cout << "----------------------------------------\n";
    std::cout << "  Throughput: " << throughput << " img/s\n";
    std::cout << "  Accuracy:   " << acc_percent << "% (" << top1 << "/" << n_test << ")\n";
    std::cout << "----------------------------------------\n";
}

int main() {
    run_gpu();
    return 0;
}

#else // __CUDACC__ not defined
int main() {
    return 1;
}
#endif // __CUDACC__
EOF

nvcc -std=c++14 -arch=sm_75 gpu_mnist_nchw.cu -O3 -o gpu_mnist_nchw

gpu_mnist_nchw.cu(29): warning #1650-D: result of call is not used
      fread(&magic, 4, 1, f);
      ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

gpu_mnist_nchw.cu(31): warning #1650-D: result of call is not used
      fread(&num_items, 4, 1, f);
      ^

gpu_mnist_nchw.cu(35): warning #1650-D: result of call is not used
          fread(&rows, 4, 1, f);
          ^

gpu_mnist_nchw.cu(37): warning #1650-D: result of call is not used
          fread(&cols, 4, 1, f);
          ^

gpu_mnist_nchw.cu(29): warning #1650-D: result of call is not used
      fread(&magic, 4, 1, f);
      ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

gpu_mnist_nchw.cu(31): warning #1650-D: result of call is not used
      fread(&num_items, 4, 1, f);
      ^

gpu_mnist_nchw.cu(35): warning #1650-D: result of call is not used
          fread(&rows, 4, 1, f);
          ^

gpu_mnist_nchw.cu(37): warning #1650-D: result of call is not used
    

In [11]:
!nvcc -arch=sm_75 gpu_mnist_nchw.cu -O3 -o gpu_mnist_nchw


gpu_mnist_nchw.cu(29): warning #1650-D: result of call is not used
      fread(&magic, 4, 1, f);
      ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

gpu_mnist_nchw.cu(31): warning #1650-D: result of call is not used
      fread(&num_items, 4, 1, f);
      ^

gpu_mnist_nchw.cu(35): warning #1650-D: result of call is not used
          fread(&rows, 4, 1, f);
          ^

gpu_mnist_nchw.cu(37): warning #1650-D: result of call is not used
          fread(&cols, 4, 1, f);
          ^

gpu_mnist_nchw.cu(29): warning #1650-D: result of call is not used
      fread(&magic, 4, 1, f);
      ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

gpu_mnist_nchw.cu(31): warning #1650-D: result of call is not used
      fread(&num_items, 4, 1, f);
      ^

gpu_mnist_nchw.cu(35): warning #1650-D: result of call is not used
          fread(&rows, 4, 1, f);
          ^

gpu_mnist_nchw.cu(37): warning #1650-D: result of call is not used
    

In [12]:
!./gpu_mnist_nchw

----------------------------------------
  Throughput: 398298 img/s
  Accuracy:   96.95% (9695/10000)
----------------------------------------


# CPU Scripts

## CPU Unoptimized

In [13]:
import torch
import numpy as np
from torchvision import datasets, transforms
import time

class NetArch(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_a = torch.nn.Conv2d(1, 8, 3, padding=1)
        self.layer_b = torch.nn.Linear(14*14*8, 10)
    def forward(self, z):
        z = torch.relu(self.layer_a(z))
        z = torch.max_pool2d(z, 2)
        z = self.layer_b(z.view(z.size(0), -1))
        return z

def tensor_from_file(path, dims):
    data = np.fromfile(path, dtype=np.float32)
    return torch.from_numpy(data.reshape(dims))

model = NetArch()
weights_a = tensor_from_file('conv1.w', (8, 3, 3, 1)).permute(0, 3, 1, 2).contiguous()
biases_a = tensor_from_file('conv1.b', (8,))
weights_b = tensor_from_file('fc.w', (10, 1568))
biases_b = tensor_from_file('fc.b', (10,))
indices = []
for i in range(14):
    for j in range(14):
        for k in range(8):
            indices.append(k*14*14 + i*14 + j)
reverse = np.argsort(indices)
weights_b2 = weights_b[:, reverse]
with torch.no_grad():
    model.layer_a.weight.copy_(weights_a)
    model.layer_a.bias.copy_(biases_a)
    model.layer_b.weight.copy_(weights_b2)
    model.layer_b.bias.copy_(biases_b)
model.eval()

transformer = transforms.Compose([transforms.ToTensor()])
data_iter = torch.utils.data.DataLoader(
    datasets.MNIST('.', train=False, download=True, transform=transformer),
    batch_size=1000, shuffle=False)

hits = count = 0
t0 = time.time()
with torch.no_grad():
    for u, v in data_iter:
        y_hat = model(u).argmax(1)
        hits += (y_hat == v).sum().item()
        count += v.size(0)
t1 = time.time()
acc = 100*hits/count
thr = count/(t1-t0)
print(f"CPU inference accuracy: {acc:.2f}% throughput: {thr:.1f} img/s")


CPU inference accuracy: 96.92% throughput: 5108.4 img/s


## CPU Optimized

In [ ]:
import torch
import numpy as np
from torchvision import datasets, transforms
import time
import os
import multiprocessing

os.environ["OMP_NUM_THREADS"] = str(multiprocessing.cpu_count())
os.environ["MKL_NUM_THREADS"] = str(multiprocessing.cpu_count())
torch.set_num_threads(multiprocessing.cpu_count())

def tensorfile(fp, shp):
    dat = np.fromfile(fp, dtype=np.float32)
    return torch.from_numpy(dat.reshape(shp))

class VariantNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.mod1 = torch.nn.Conv2d(1, 8, 3, padding=1)
        self.mod2 = torch.nn.Linear(14*14*8, 10)
    def forward(self, z):
        if z.dtype != torch.float32:
            z = z.float()
        if z.dim() == 3:
            z = z.unsqueeze(0)
        y = torch.relu(self.mod1(z))
        y = torch.max_pool2d(y, 2)
        y = self.mod2(y.view(y.size(0), -1))
        return y

def run_cpu():
    cpus = multiprocessing.cpu_count()
    mdl = VariantNet()
    t_init = time.time()
    w_a = tensorfile('conv1.w', (8, 3, 3, 1)).permute(0, 3, 1, 2).contiguous()
    b_a = tensorfile('conv1.b', (8,))
    w_b = tensorfile('fc.w', (10, 1568))
    b_b = tensorfile('fc.b', (10,))
    idxs = [(c*14*14 + h*14 + w) for h in range(14) for w in range(14) for c in range(8)]
    rev = np.argsort(idxs)
    w_b2 = w_b[:, rev]
    with torch.no_grad():
        mdl.mod1.weight.copy_(w_a)
        mdl.mod1.bias.copy_(b_a)
        mdl.mod2.weight.copy_(w_b2)
        mdl.mod2.bias.copy_(b_b)
    scripted_net = torch.jit.script(mdl)
    scripted_net.eval()
    tfm = transforms.Compose([transforms.ToTensor()])
    batch_sizes = list(range(1000, 10001, 1000))
    for N in batch_sizes:
        loader = torch.utils.data.DataLoader(
            datasets.MNIST('.', train=False, download=True, transform=tfm),
            batch_size=N, shuffle=False, pin_memory=True,
            num_workers=min(4, cpus), persistent_workers=True)
        total = match = 0
        t1 = time.time()
        for u, _ in torch.utils.data.DataLoader(
                datasets.MNIST('.', train=False, transform=tfm), batch_size=10):
            with torch.no_grad():
                scripted_net(u)
            break
        with torch.no_grad():
            for u, v in loader:
                pred = scripted_net(u).argmax(1)
                match += (pred == v).sum().item()
                total += v.size(0)
        t2 = time.time()
        acc = 100*match/total
        thr = total/(t2-t1)
        print(f"CPU inference accuracy: {acc:.2f}% throughput: {thr:.1f} img/s")

if __name__ == "__main__":
    run_cpu()


CPU inference accuracy: 96.92% throughput: 5696.2 img/s
CPU inference accuracy: 96.92% throughput: 4951.0 img/s
CPU inference accuracy: 96.92% throughput: 3414.0 img/s
CPU inference accuracy: 96.92% throughput: 4010.8 img/s
CPU inference accuracy: 96.92% throughput: 5105.2 img/s
